# Task 1. Vigenère Algorithm

In [400]:
KEYWORD = "KEY"
SEQ_LEN = 3
MAX_KEY_LEN = 20
EN_REL_FREQ = {'A': 0.08167, 'B': 0.01492, 'C': 0.02782, 'D': 0.04253, 'E': 0.12702, 'F': 0.02228, 'G': 0.02015,
               'H': 0.06094, 'I': 0.06966, 'J': 0.00153, 'K': 0.00772, 'L': 0.04025, 'M': 0.02406, 'N': 0.06749,
               'O': 0.07507, 'P': 0.01929, 'Q': 0.00095, 'R': 0.05987, 'S': 0.06327, 'T': 0.09056, 'U': 0.02758,
               'V': 0.00978, 'W': 0.02360, 'X': 0.00150, 'Y': 0.01974, 'Z': 0.00074}
ALPHABET = list(EN_REL_FREQ.keys())

In [401]:
string_to_encrypt = "CRYPTOGRAPHYANDDATASECURITY"

In [402]:
from collections import defaultdict


def _make_encrypt_table(alphabet: list[str], key_word: str):
    encrypt_table = defaultdict(list)
    for ch in key_word:
        dist = ord(ch) - ord(alphabet[0])
        encrypt_table[ch] = alphabet[dist:] + alphabet[:dist]
    return encrypt_table

In [403]:
def _full_string_with_keyword(s: str, key_word: str):
    key_line = key_word * int(len(s) / len(key_word))
    ost = int(len(s) % len(key_word))
    key_line += key_word[:ost]
    return key_line

In [404]:
def encrypt_string(s: str, alphabet: list[str] = ALPHABET, key_word: str = KEYWORD):
    s = s.upper()
    encrypt_table = _make_encrypt_table(alphabet, key_word)
    key_line = _full_string_with_keyword(s, key_word)
    encrypted_str = ""
    for ind in range(len(key_line)):
        if s[ind] not in alphabet:
            encrypted_str += s[ind]
            continue
        pos_in_alphabet = alphabet.index(s[ind])
        encrypted_str += encrypt_table[key_line[ind]][pos_in_alphabet]
    return encrypted_str

In [405]:
encrypted_string = encrypt_string(string_to_encrypt)
encrypted_string

'MVWZXMQVYZLWKRBNERKWCMYPSXW'

## Encrypting large strings

In [406]:
needed_lines_length = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000]
lines = []
data_path = '../data/harry_potter.txt'

In [407]:
with open(data_path, 'r') as file:
    s = file.read()
    s = ''.join(char for char in s if char.isalnum()).upper()
    prev_str_end = 0
    for length in needed_lines_length:
        lines.append(s[prev_str_end:length])
        prev_str_end = length

In [408]:
encrypted_lines = [
    encrypt_string(line) for line in lines
]

# Decryption

## Helper functions

In [409]:
def get_blocks(text, size):
    blocks = [text[i:i + size] for i in range(0, len(text) - size, size)]
    return blocks


def get_columns(text_blocks):
    group_size = len(text_blocks[0])
    columns = []
    for letter_count in range(group_size):
        column = ''
        for group_count in range(len(text_blocks)):
            column += text_blocks[group_count][letter_count]
        columns.append(column)
    return columns


def to_blocks(cols):
    col_size = len(cols[0])
    blocks = []
    for letter in range(col_size):
        block = ''
        for col in range(len(cols)):
            block += cols[col][letter]
        blocks.append(block)
    return blocks

## Find keyword's len with Kasiski method

In [410]:
from math import sqrt


def _repeated_seq_pos(text, seq_len):
    seq_pos = {}  # entries of sequence : [positions]
    for i, char in enumerate(text):
        next_seq = text[i:i + seq_len]
        if next_seq in seq_pos.keys():
            seq_pos[next_seq].append(i)
        else:
            seq_pos[next_seq] = [i]
    repeated = list(filter(lambda x: len(seq_pos[x]) >= 2, seq_pos))
    rep_seq_pos = [(seq, seq_pos[seq]) for seq in repeated]
    return rep_seq_pos


def _get_spacings(positions):
    return [positions[i + 1] - positions[i] for i in range(len(positions) - 1)]


def _get_factors(number):
    factors = set()
    for i in range(1, int(sqrt(number)) + 1):
        if number % i == 0:
            factors.add(i)
            factors.add(number // i)
    return sorted(factors)


def _candidate_key_lengths(factor_lists, max_key_len):
    all_factors = [factor_lists[lst][fac] for lst in range(len(factor_lists)) for fac in range(len(factor_lists[lst]))]
    # exclude factors larger than suspected max key length
    candidate_lengths = list(filter(lambda x: 1 < x <= max_key_len, all_factors))
    # sort by probability (descending)
    sorted_candidates = sorted(set(candidate_lengths), key=lambda x: all_factors.count(x), reverse=True)
    return sorted_candidates


def find_key_length(cyphertext, seq_len, max_key_len):
    # find repeated sequences and their positions
    rsp = _repeated_seq_pos(text=cyphertext, seq_len=seq_len)
    seq_spc = {}
    for seq, positions in rsp:
        seq_spc[seq] = _get_spacings(positions)
    # calculate spacings between positions of each repeated
    # sequence and factor out spacings
    factor_lists = []
    for spacings in seq_spc.values():
        for space in spacings:
            factor_lists.append(_get_factors(number=space))
    # get common factors by descending frequency,
    # which constitute candidate key lengths
    ckl = _candidate_key_lengths(factor_lists=factor_lists, max_key_len=max_key_len)
    if len(ckl) == 0:
        raise ValueError("Could not attack provided cyphertext")
    key_len = ckl[0]
    return key_len

## Guessing keywords with its length

In [411]:
import string


def get_letter_counts(text):
    text_upper = text.upper()
    letter_counts = {}
    for index, letter in enumerate(string.ascii_uppercase):
        letter_counts[letter] = text_upper.count(letter)
    return letter_counts


def _get_letter_frequencies(text):
    letter_counts = get_letter_counts(text)
    frequencies = {letter: count / len(text) for letter, count in letter_counts.items()}
    return frequencies


def shift(text, amount):
    shifted = ''
    letters = string.ascii_uppercase
    for letter in text:
        shifted += letters[(letters.index(letter) - amount) % len(letters)]
    return shifted


def _corr(text, lf):
    return sum([(lf[letter] * EN_REL_FREQ[letter]) for letter in text])


def _find_key_letter(text, lf):
    key_letter = ''
    max_corr = 0
    for count, letter in enumerate(string.ascii_uppercase):
        shifted = shift(text=text, amount=count)
        corr = _corr(text=shifted, lf=lf)
        if corr > max_corr:
            max_corr = corr
            key_letter = letter
    return key_letter


def restore_key(cyphertext, key_len):
    key = ''
    blocks = get_blocks(text=cyphertext, size=key_len)
    columns = get_columns(blocks)
    frequencies = _get_letter_frequencies(text=cyphertext)
    for column in columns:
        key += _find_key_letter(text=column, lf=frequencies)
    return key

In [412]:
def decypher(cyphertext, key):
    letters = string.ascii_uppercase
    shifts = [letters.index(letter) for letter in key]
    blocks = get_blocks(text=cyphertext, size=len(key))
    cols = get_columns(blocks)
    decyphered_blocks = to_blocks([shift(col, shiftt) for col, shiftt in zip(cols, shifts)])
    decyphered = ''.join(decyphered_blocks)
    return decyphered

In [413]:
def attack(cyphertext: str):
    key_len = find_key_length(cyphertext=cyphertext, seq_len=SEQ_LEN, max_key_len=MAX_KEY_LEN)
    print(key_len)
    key = restore_key(cyphertext, key_len)
    decyphered = decypher(cyphertext, key)
    print('Chosen key length: ' + str(key_len))
    print('Restored key: ' + str(key))
    print('Plaintext: ' + str(decyphered))

In [414]:
attack(encrypted_lines[-2])

3
Chosen key length: 3
Restored key: KEY
Plaintext: THENEIGHBORSWOULDSAYIFTHEPOTTERSARRIVEDINTHESTREETTHEDURSLEYSKNEWTHATTHEPOTTERSHADASMALLSONTOOBUTTHEYHADNEVEREVENSEENHIMTHISBOYWASANOTHERGOODREASONFORKEEPINGTHEPOTTERSAWAYTHEYDIDNTWANTDUDLEYMIXINGWITHACHILDLIKETHATWHENMRANDMRSDURSLEYWOKEUPONTHEDULLGRAYTUESDAYOURSTORYSTARTSTHEREWASNOTHINGABOUTTHECLOUDYSKYOUTSIDETOSUGGESTTHATSTRANGEANDMYSTERIOUSTHINGSWOULDSOONBEHAPPENINGALLOVERTHECOUNTRYMRDURSLEYHUMMEDASHEPICKEDOUTHISMOSTBORINGTIEFORWORKANDMRSDURSLEYGOSSIPEDAWAYHAPPILYASSHEWRESTLEDASCREAMINGDUDLEYINTOHISHIGHCHAIRNONEOFTHEMNOTICEDALARGETAWNYOWLFLUTTERPASTTHEWINDOWATHALFPASTEIGHTMRDURSLEYPICKEDUPHISBRIEFCASEPECKEDMRSDURSLEYONTHECHEEKANDTRIEDTOKISSDUDLEYGOODBYEBUTMISSEDBECAUSEDUDLEYWASNOWHAVINGATANTRUMANDTHROWINGHISCEREALATTHEWALLSLITTLETYKECHORTLEDMRDURSLEYASHELEFTTHEHOUSEHEGOTINTOHISCARANDBACKEDOUTOFNUMBERFOURSDRIVEITWASONTHECORNEROFTHESTREETTHATHENOTICEDTHEFIRSTSIGNOFSOMETHINGPECULIARACATREADINGAMAPFORASECONDMRDURSLEYDIDNTREALIZEWHATHEH